# Async LLM, LLMChain, Agent

Many operations in LangChain are i/o bound. Running concurrently offers significant advantages.

## Async LLMChain

In [14]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


def generate_serially():
    llm = OpenAI(temperature=0.9)
    prompt = PromptTemplate(
        input_variables=["product"],
        template="What is a good name for a company that makes {product}?",
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    for _ in range(5):
        resp = chain.run(product="toothpaste")
        print(resp)


async def async_generate(chain):
    resp = await chain.arun(product="toothpaste")
    print(resp)


async def generate_concurrently():
    llm = OpenAI(temperature=0.9)
    prompt = PromptTemplate(
        input_variables=["product"],
        template="What is a good name for a company that makes {product}?",
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    tasks = [async_generate(chain) for _ in range(5)]
    await asyncio.gather(*tasks)


import time

s = time.perf_counter()
await generate_concurrently()
elapsed = time.perf_counter() - s
print(f"Concurrent executed in {elapsed:0.2f} seconds.")

s = time.perf_counter()
generate_serially()
elapsed = time.perf_counter() - s
print(f"Serial executed in {elapsed:0.2f} seconds.")



BrightSmile Toothpaste


FreshSmilez Inc.


BrightSmile Toothpaste.


Bright White Toothpaste Co.


BrightSmile Toothpaste.
Concurrent executed in 1.62 seconds.


MintyFresh Toothpaste Co.


FreshSmile Toothpaste Company


SmileFresh Inc.


BrightSmile Toothpaste Company


Bright Smile Toothpaste Company
Serial executed in 3.15 seconds.


## Async Agent

In [7]:
import langchain
from langchain.agents import Tool, initialize_agent, load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
tools = load_tools(["llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent="zero-shot-react-description", verbose=True
)


async_tools = load_tools(["async-llm-math"], llm=llm)
async_agent = initialize_agent(
    async_tools, llm, agent="zero-shot-react-description", verbose=True
)

In [8]:
agent.run("What is 2 raised to .123243 power?")



> Entering new AgentExecutor chain...
 I need to use a calculator to solve this.
Action: Calculator
Action Input: 2^.123243
Observation: Answer: 1.0891804557407723

Thought: I now know the final answer.
Final Answer: 1.0891804557407723

> Finished chain.


'1.0891804557407723'

In [11]:
await async_agent.arun("What is 2 raised to .123243 power?")



> Entering new AgentExecutor chain...
 I need to use a calculator to solve this.
Action: Calculator (Async)
Action Input: 2^.123243
Observation: Answer: 1.0891804557407723

Thought: I now know the final answer.
Final Answer: 1.0891804557407723

> Finished chain.


'1.0891804557407723'

In [12]:
def generate_serially(agent):
    for _ in range(5):
        agent.run("What is 2 raised to .123243 power?")


async def generate_concurrently(async_agent):
    tasks = [async_agent.arun("What is 2 raised to .123243 power?") for _ in range(5)]
    await asyncio.gather(*tasks)



import time

s = time.perf_counter()
await generate_concurrently(async_agent)
elapsed = time.perf_counter() - s
print(f"Concurrent executed in {elapsed:0.2f} seconds.")

s = time.perf_counter()
generate_serially(agent)
elapsed = time.perf_counter() - s
print(f"Serial executed in {elapsed:0.2f} seconds.")



> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...
 I need to use a calculator to solve this.
Action: Calculator (Async)
Action Input: 2^.123243 I need to use a calculator to solve this.
Action: Calculator (Async)
Action Input: 2^.123243 I need to use a calculator to solve this.
Action: Calculator (Async)
Action Input: 2^.123243 I need to use a calculator to solve this.
Action: Calculator (Async)
Action Input: 2^.123243 I need to use a calculator to solve this.
Action: Calculator (Async)
Action Input: 2^.123243
Observation: Answer: 1.0891804557407723

Thought:
Observation: Answer: 1.0891804557407723

Thought:
Observation: Answer: 1.0891804557407723

Thought: I now know the final answer.
Final Answer: 1.0891804557407723

> Finished chain.

Observation: Answer: 1.0891804557407723

Thought:
Observation: Answer: 1.0891804557407723

Thought: I 